# Scikic API v9.0

## Change log
### Changes since v8.0

 - Added audience estimation
 
### Changes since v7.0

 - Added Green Simple insight
 
### Changes since v6.0

 - Added configuration option to inference to speed up MCMC, defaults to fast and poor inference
 - Added smoothing to try to hide some of the poor quality of the inference in the age distribution.
 
### Changes since v5.0

 - Added relationships to the API's inference output (back compatible!)

### Changes since v4.0

 - The metadata command takes a list for dataset, rather than an individual dataset.
        data = {'dataset':['ukcensus','uscensus']}

### Changes since v2.0

 - psychometrics from Cambridge now have new structure (and return percentiles)
 - insights is now a DICTIONARY not a list. So you can access particular answers.
 - inference can contain a list of dataset classes that it should use. If this is not passed it will use all the classes it can.
 
## Overview

The scikic api is an inference tool which takes a set of question/answer items and then queries a series of local and remote databases to generate conditional probability distributions over various features. The api is highly modular, and some modules don't use this probabilistic framework, for example the music module simply contacts api.bandsintown.com to provide useful suggestions about local bands to go and see.

The conditional probabilities are combined using a Bayesian network, using the pyMC module. Each module can provide pyMC 'features' which create functions to output the relevant probability distributions.

## Question/Answer dictionaries

The questions and answers are organised to be in 4 value tuples, containing:

- dataset: lets the system know which class to instantiate etc, examples: postcode, census, movielens, ...etc
- dataitem: used by classes to know which aspect of the dataset. For example in the movielens dataset, one could be interested in whether the user's seen a film or what rating they've given the film.
- detail: often unused by the classes, could be, for example, the id of the film we want to know about.
- answer: the user's answer.

## Future Changes/TODO list

Heads-up about a couple of changes:

 - ~~Vasily suggested I move the 'action' selection out and make it part of the URL, to make it more RESTful.~~ DONE.
 - ~~I'll be combining the API query that gets the question string, with the one that generates the 'raw' question tuple. Originally the API was designed when it was run on the same server as the code that was querying it, so extra API calls didn't cost much. As the calls will now be remote, they need combining.~~ DONE
 - ~~document the new API~~ DONE (below)
 - fix security issues (stop outputting debug info to client)
 - add useful error messages to API's output
 - incorporate Cambridge's code
 - write a new install script for the setup
 - install on front end
 - The version number will be used in future.
 - The API doesn't currently check the key. Obviously this will be implemented in future.
 - Get the IP->Location working again (API change?)
 - Check load balancer to backend API still working (this shouldn't be need altering)
 - Set up SSL for API load balancer (previously left connection between frontend and backend non-SSL as it's all in the AWS datacentre).

## Available actions

Here are some examples of the API in action. A **POST** request is used for the query, in case the data we're sending is too large to fit in a GET request. The POST body is json.

### 1. question

#### Parameters
One passes to this call in data, a dictionary containing:
 - 'questions_asked'
 - 'unprocessed_questions'
 - 'facts'
 - 'target'

The 'questions_asked' include all the questions we've asked, so we don't ask the same question again.
The 'facts' dictionary contains information that we've found from earlier questions, etc. It allows caching of the calculations from earlier calls to the API.
The 'unprocessed_questions' are a list of question/answers that we've asked before, that haven't had their results added to the 'facts' dictionary.
The 'target' item is currently unused, but in the future will allow the choice of question to be selected to maximise the information about a particular feature.

#### Returns

This call returns a dictionary containing three things:

 - a 'facts' dictionary - this you can pass back in future so that the method doesn't have to recalculate or generate earlier results.
 - a 'question' dictionary, containing the dictionary describing the question, e.g. {dataset,dataitem,detail}.
 - a 'question_string' dictionary, containing:
 
    - 'text' - the actual string of the question (e.g. "Who's your favourite band or artist?")
    - 'type' - the type of question (it might just want a text reply, so this would equal 'text' or it might be a choice, and so would say 'select'
    - 'options' - optional, and is included if the type is 'select'.


#### Usage example

1. One might initially call this method with all these fields being empty. The method will return an empty 'facts' dictionary and a question dictionary for the first question you want to ask. 
2. Once you have an answer from the user you would call the method again, this time with the question/answer tuple as both 'questions_asked' and 'unprocessed_questions'.
3. The method will return a facts dictionary now, potentially with some results from the processing of the last answer, and another question for you to ask. 
4. When you call the method a third time (with the user's second answer), you'll pass all the question/answer tuples that you've asked so far in 'questions_asked' and the last question/answer tuple in 'unprocessed_questions'. You'll also pass the new facts dictionary, that now has some content in it.
5. This process continues, with the facts dictionary growing each time, the 'questions_asked' growing too, and each time you just have one item in 'unprocessed_questions'.

To summarise:

Generating a question requires a dictionary of 'questions_asked', 'facts' and 'target'. The 'questions_asked' is a list of dictionaries of previous questions, that you want to avoid asking again.
The 'unprocessed_questions' are questions that you've asked already and that haven't been incorporated into the 'facts' dictionary.

Below is an example:

In [2]:
#The url of the API currently (in the long run we should give the backend a domain, maybe api.scikic.org?)
apiurl = 'http://api.scikic.org' #production api<<<
#apiurl = 'http://dev.scikic.org' #development api
#apiurl = 'http://54.194.2.173' #backend image server

In [3]:
import requests
#We provide data about previous questions etc:
#data consists of a dictionary of (all of these are optional):

#'questions_asked': An array of previous questions and answers we've asked, consists of a list of dictionaries.
questions_asked = [{'dataset':'postal','dataitem':'postcode','detail':''},{"detail": "", "dataitem": "favourite_artist", "dataset": "music"}]

#none of the questions that have been asked before have been processed.
unprocessed_questions = questions_asked

#'facts': If you've run the inference query and stored a copy of the facts dictionary you can pass it back.
#this makes it quicker. In this case we've not yet had any questions processed.
facts = {}

#'target': What feature we want to know more about (example: 'age', 'gender', 'location') NOT YET IMPLEMENTED
#not used. All these items are optional, so we just don't include it.

#Build the dictionary:
data = {'unprocessed_questions':unprocessed_questions,'questions_asked':questions_asked,'facts':facts}

#put it into the payload of the request. This also includes the version and the api key we're using
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/question',json=payload)
r.content

'{"facts": {"guess_loc": {}, "where": {}}, "question_string": {"type": "select", "question": "Which country are you in now?", "options": ["United States", "United Kingdom", "Germany", "Canada", "France", "Other"]}, "question": {"detail": "{}", "dataitem": "country", "dataset": "geoloc"}}'

Here it has output:

 - the facts dictionary: Empty
 - the question it suggests we ask, which in an earlier case is from the movielens dataset class, and is asking if they've seen movie number 2541.
 - the question string, with the type of question and options (if relevant). For example the dictionary `{'dataset':'postal','dataitem':'postcode','detail':''}` gets converted to the question "`What's your postcode?`" with type "`text`" (i.e. the user can type anything).

### 2. inference

#### Parameters

The data dictionary should contain three things, similar to the "action: question" above,

- questions_asked - list of question tuples that we've asked (with their answers)
- unprocessed_questions - list of question tuples that we've asked (with their answers), that have not yet been added to the facts dictionary.
- facts - the current 'facts' dictionary (possibly provided by earlier calls using action:question)

**NEW** It optionally can contain a list of dataset classes that it should use. If this is not passed it will use all the classes it can.

- datasets - list of datasets.

#### Output

Returns a dictionary of:

##### features
This is a dictionary of things that have probabilities associated, for example one of its items is 'household' with the following fields:
 
     {"distribution": [0.029, 0.058, 0.23, 0.034, 0.070, 0.026, 0.055, 0.036, 0.14, 0.024, 0.023, 0.035, 0.24], "quartiles": {"upper": 11, "lower": 2, "mean": 6.46}}
 
 where the distribution is how likely the person is to be in each of the categories of a household (these categories can be found in the metadata from the module, or elsewhere). The quartiles don't mean much here as this is properly categorical data. This makes more sense in data such as age.
 
##### facts
As mentioned previously is a set of truths about the user generated from processing their answers.

##### Insights
** CHANGE **
This is now a dictionary of strings, generated by each module, here is an example:
 
     {u'ukcensus_traveltowork': u'People in your area are 3 times more likely to go to work on foot than the national average.', u'ukcensus_household': u"You don't have children living at home", u'ukcensus_ages': u'You are aged between 21 and 34.', u'ukcensus_religion': u' I think you are Christian or of no religion.', u'ukcensus_popage': u'Half the people in your neighbourhood are younger than 25 years old.', u'ukcensus_popdensity': u'Your neighbourhood has a population density of 1930 people per square kilometre, 4.7 times the average for England.'}
 
Note that each string is identified by a label in the dictionary, allowing easier access to the strings by the client.

*Note regarding the distribution above: If some probabilities are zero towards the end of a list then the list will be truncated. For example if inference is certain the user is a male, then the output list will be {"factor_gender":[1.0]}. If they are definitely female it will be {"factor_gender":[0.0, 1.0]}*

##### Relationships
** CHANGE **
Allows the graphical model to be displayed on the web conversational interface

#### Example call using action:inference

Below we set up the data dictionary with one question asked (and answered) regarding postcode. It also contains the same question/answer dictionary as an unprocessed question. The facts dictionary is empty.

The output

** THE UK ONS HAS WITHDRAWN API ACCESS THE FOLLOW EXAMPLE NO LONGER WORKS **

In [ ]:
import requests
import json
from timeit import default_timer as time

postcode = 's63af'
questions_asked = []

#their address:
#questions_asked.append({'dataset':'postal','dataitem':'postcode','detail':'','answer':'kuhdrfliudrh'})
questions_asked.append({'dataset':'postal','dataitem':'postcode','detail':'','answer':'s63af'})
#questions_asked.append({'dataset':'geoloc','dataitem':'country','detail':'','answer':'United Kingdom'})

#Additional info from facebook:
#fb_data = json.dumps({'reply':json.dumps({'first_name':'Alan'})})
#questions_asked.append({'dataset':'facebook','dataitem':'','detail':'','answer':fb_data})

datasets = ['demographic','postal','ukcensus','uscensus','babynames']
#we assume that we've not processed any of these questions yet. 
#You'll only not do this if you've asked some of these already,
#and populated the 'facts' dictionary.
unprocessed_questions = questions_asked[:]
facts = {}
configuration = {'mcmc_iterations':100} #50,000 probably the most you'd want here. defaults to 1,000
data = {'questions_asked':questions_asked,
        'unprocessed_questions':unprocessed_questions,
        'facts':facts,
        'config':configuration}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}

t0 = time()
r = requests.post(apiurl+'/inference',json=payload)
t1 = time()
print "TOTAL TIME %0.0fms" % (1000*(t1-t0))
result = json.loads(r.content)
print "==Facts=="
print result['facts']
print ""
print "==Features=="
for feat in result['features']:
    print feat
    print result['features'][feat]
print ""
print "==Text insights=="
print result['insights']
print "==Relationships=="
print result['relationships']

For the example above one can see that after just processing the answer about postcode, quite a bit of new info has been generated.

First the facts dictionary. This is often dataset specific stuff, although I've tried to make things compatable between datasets.

 - 'guess_loc': {} - info on whether it guessed the location of the user from IP address (I think)
 - 'where': - the dictionary describing the location of the user's home. This is quite tricky, as different sources of data have different resolutions, etc about this.
     - {'ukcensus': [{'item': 'E00172420', 'probability': 1.0, 'level': 'oa'}] - In terms of the UK census, with have a list of output areas for this person's home. This list only has one item in it. Each item in the list has a probability associated, in this case the probability is 1.0: We are certain the person is in that output area.
     - 'country': [{'item': 'gb', 'probability': 1.0}] - Which country they're in. A list of countries with associated probabilities.
     - 'city': [{'item': ['Sheffield', 'uk'], 'probability': 1.0}]} - which city their in (with probabilites).
     - 'where_history': {u'error': u'no_fb_likes'}} - if we have access to facebook likes, it tries to generate a history of where the person's lived. But the error item means that it's not managed to get hold of the likes to do this.

Next the features dictionary. Different datasets provide different conditional probability distributions. Each distribution has at least two features associated. If they are not in the list already the module adds them, thus one has a list of features at the end.

The value of these features is then estimated using MCMC with the pyMC module.

The example above has five features: religion, household, (census) output area, gender and age. They're all catagorical (for now) although that's purely due to the type of data that we have about them.

Looking just at the household feature: [0.022222222222222223, 0.05688888888888889, 0.23466666666666666, 0.03244444444444444, 0.08266666666666667, 0.028, 0.059111111111111114, 0.03955555555555555, 0.14222222222222222, 0.024, 0.024444444444444446, 0.028444444444444446, 0.22533333333333333]

Each value refers to the probability of being of a given type of household (the labels associated are available via the metadata of the module).

Finally there are insights, these are simple strings of facts about the person.
 
Here's another example, with an american zip code:

<a id="usexample" />

In [5]:
import requests
import json
from timeit import default_timer as time

questions_asked = []

#their address:
#questions_asked.append({'dataset':'postal','dataitem':'zipcode','detail':'','answer':'86020'})
questions_asked.append({'dataset':'postal','dataitem':'zipcode','detail':'','answer':'90210'})
questions_asked.append({'dataset':'demographic','dataitem':'age','detail':'','answer':'33'})

datasets = ['demographic','postal','ukcensus','uscensus']
#we assume that we've not processed any of these questions yet. 
#You'll only not do this if you've asked some of these already,
#and populated the 'facts' dictionary.
unprocessed_questions = questions_asked[:]
facts = {}
data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts,'datasets':datasets}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}

t0 = time()
r = requests.post(apiurl+'/inference',json=payload)
t1 = time()
print "TOTAL TIME %0.0fms" % (1000*(t1-t0))

print r.content

result = json.loads(r.content)
print "==Facts=="
print result['facts']
print ""
print "==Features=="
for feat in result['features']:
    print feat
    print result['features'][feat]
print ""
print "==Text insights=="
print result['insights']

TOTAL TIME 119ms
{"facts": {"age": 33}, "relationships": [], "feature_descriptions": {"factor_gender": {"desc": "Your gender"}, "factor_age": {"desc": "Your age"}}, "features": {"factor_age": {"distribution": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379], "quartiles": {"upper": 33, "lower": 33, "mean": 33.0}}}, "insights": {"debug_total_inference_time": "53.290844ms"}}
==Facts==
{u'age': 33}

==Features==
factor_age
{u'distribution': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.03162277660168379, 0.0316

You can see this is similar, except the 'where' item in the dictionary has a 'uscensus' item within it. This contains two items:

    {"item": ["04", "015", "950100", "1"], "probability": 0.514, "level": "blockgroup"}
    {"item": ["04", "015", "950100", "3"], "probability": 0.486, "level": "blockgroup"}

Because zipcodes cover quite large areas, it doesn't know which blockgroup the person's home is in, as the zip code spans more than one blockgroup. It therefore gives the probability of being in the two.

The US census module doesn't know about religion, so doesn't have a conditional probability distribution about it, so no religion feature is created. The features that are created are:

 - bg (block group)
 - gender
 - age

### 3. Metadata *[action: metadata]*

Some of the classes provide metadata about the results. Use the 'metadata' action to retrieve these. Pass a dictionary in 'data' with the names of the datasets, or leave empty to get all the metadata of all the classes.

Change in v5: The metadata takes a list for dataset, rather than an individual dataset.
        
    data = {'dataset':['ukcensus','uscensus']}

In this example we display the citation information for the 'babynames' dataset.

In [11]:
import requests
import json
data = {'dataset':['uscensus']}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'citation' in item:
        print(item['citation'])

The <a href="http://www.census.gov/developers/">US census bureau</a>. In particular the American Community Survey <a href="http://www.census.gov/data/developers/data-sets/acs-survey-5-year-data.html">5 year data</a>.


In this example we get the list of languages supported by the uk census

In [12]:
import requests
import json
data = {'dataset':['ukcensus','uscensus']}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'languages_text' in item:
        print(item['languages_text'])

[u'English', u'Spanish', u'French', u'French Creole', u'Italian', u'Portuguese or Portuguese Creole', u'German', u'Yiddish', u'a Scandinavian language', u'Greek', u'Russian', u'Polish', u'Serbo-Croatian', u'Armenian', u'Persian', u'Gujarati', u'Hindi', u'Urdu', u'Chinese', u'Japanese', u'Korean', u'Mon-Khmer, Cambodian', u'Hmong', u'Thai', u'Laotian', u'Vietnamese', u'Tagalog', u'Navajo', u'Hungarian', u'Arabic', u'Hebrew', u'an African language']
[u'Afrikaans', u'Akan', u'Amharic', u'Igbo', u'Krio', u'Lingala', u'Luganda', u'Shona', u'Somali', u'Swahili', u'Tigrinya', u'Yoruba', u'Arabic', u'Caribbean Creole', u'Cantonese Chinese', u'Japanese', u'Korean', u'Malay', u'Mandarin Chinese', u'Tagalog/Filipino', u'Thai', u'Vietnamese', u'English', u'French', u'Bulgarian', u'Czech', u'Danish', u'Dutch', u'Estonian', u'Finnish', u'German', u'Greek', u'Hungarian', u'Italian', u'Latvian', u'Lithuanian', u'Maltese', u'Polish', u'Romanian', u'Slovak', u'Slovenian', u'Swedish', u'Albanian', u'Serb

In this example we get a list of all the datasets available.

In [29]:
import requests
import json
data = {}#no dataset specified (makes it output all metadata)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'citation' in item:
        print(item['dataset'])

babynames
music
uscensus
geoloc
facebook
postal
ukcensus
movielens


In this example we get all citations:

In [5]:
import requests
import json
data = {}#no dataset specified (makes it output all metadata)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'citation' in item:
        print(item['citation'])

The <a href="http://files.grouplens.org/datasets/movielens">movielens</a> database
The <a href="http://www.ons.gov.uk/ons/guide-method/census/2011/census-data/ons-data-explorer--beta-/index.html">UK office of national statistics</a>
The ONS provide statistics on the distribution of the names of baby's in the UK: <a href="http://www.ons.gov.uk/ons/about-ons/business-transparency/freedom-of-information/what-can-i-request/published-ad-hoc-data/pop/august-2014/baby-names-1996-2013.xls">1996-2013</a> and <a href="http://www.ons.gov.uk/ons/rel/vsob1/baby-names--england-and-wales/1904-1994/top-100-baby-names-historical-data.xls">1904-1994</a>.
The <a href="facebook.com">facebook</a> graph API
The <a href="http://www.census.gov/developers/">US census bureau</a>. In particular the American Community Survey <a href="http://www.census.gov/data/developers/data-sets/acs-survey-5-year-data.html">5 year data</a>.
The <a href="freegeoip.net">freegeoip.net</a> API
The <a href="https://geoportal.statist

In this example we get the household types:

In [10]:
import requests
import json
data = {'dataset':['ukcensus','uscensus']}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    print '===='+item['dataset']+'==='
    if 'households_text' in item:
        print "households text"
        print item['households_text']
    if 'households_census_labels' in item:
        print "household census labels"
        print item['households_census_labels']

====ukcensus===
households text
[u'Cohabiting couple (children have left home)', u'Cohabiting couple with children', u'Cohabiting couple, without children', u'Single person (children have left home)', u'Lone parent', u'Married couple (children have left home)', u'Married couple with children', u'Married couple, without children', u'Single person', u'Other households, with children', u'Retired couple', u'Retired single person', u'Students and retired']
household census labels
[u'One family only: Cohabiting couple: All children non-dependent', u'One family only: Cohabiting couple: Dependent children', u'One family only: Cohabiting couple: No children', u'One family only: Lone parent: All children non-dependent', u'One family only: Lone parent: Dependent children', u'One family only: Married or same-sex civil partnership couple: All children non-dependent', u'One family only: Married or same-sex civil partnership couple: Dependent children', u'One family only: Married or same-sex civil pa

In this code we're just getting all the metadata:

In [7]:
import requests
import json
data = {'dataset':['ukcensus','uscensus']}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for items in json.loads(r.content):
    print "\n\n-----------------------------------"
    print items['dataset']
    for item in items:
        print " "
        print item
        print items[item]



-----------------------------------
uscensus
 
households_text
[u'All family household', u'Married couple', u'Other family household', u'Single dad', u'Single mum', u'Non-family household', u'Single person', u'Shared property']
 
birthplace_text
[u'Born in state of residence', u'Born in other state in Northeast US', u'Born in other state in Midwest US', u'Born in other state in South US', u'Born in other state in West US', u'Born outside the US']
 
birthplace_codes
[u'B05002_003E', u'B05002_005E', u'B05002_006E', u'B05002_007E', u'B05002_008E', u'B05002_013E']
 
households_codes
[u'B11001_002E', u'B11001_003E', u'B11001_004E', u'B11001_005E', u'B11001_006E', u'B11001_007E', u'B11001_008E', u'B11001_009E']
 
_states
[u'Mo29', u'Al01', u'Ak02', u'Az04', u'Ar05', u'Ca06', u'Co08', u'Ct09', u'De10', u'Dc11', u'Fl12', u'Ga13', u'Hi15', u'Id16', u'Il17', u'In18', u'Ia19', u'Ks20', u'Ky21', u'La22', u'Me23', u'Md24', u'Ma25', u'Mi26', u'Mn27', u'Ms28', u'Mt30', u'Ne31', u'Nv32', u'Nh33', u'

### 4. Psychometrics, provided by Cambridge

This calls the psychometric analysis, provided by Cambridge.

Pass in a string called 'userstatus'. The API returns a directionary of two dictionaries, one called "percentiles" and one called "scores".

Each of these has five items 'ope', 'con', 'ext', 'agr', 'neu', which relate to the big5 personality features.

In [64]:
import requests

userstatus = """ LOL! 'Take a deep breath\n2. think of someone u like\n3. press F10 5 times\n4. send this to 5 coments\n5. look at ur background\n9 minutes ago ', 'Rotate your facebook friends omg =).  http://tinyurl.com/37wdxok', 'sarap!!!!!!!!!!!!!!!!!1', '???? Happy New Year In Advance!! ????? ?????????????????????????????????? ????????????????????????????????? ???????????????????????????? ????? ?????????????????????????????????? ?????????????????????????????????? ?????????????????????????????????? ??????????????????????????????????', 'Rotate your facebook friends omg =).  http://tinyurl.com/2ug8r4c', "Apple finally gives away 100 Iphone 4's\n http://apps.facebook.com/qbquiz-ghflb", 'ano susuutin bukas ??????', 'Rotate your facebook friends omg =).  http://tinyurl.com/3xngp4s', 'Rotate your facebook friends omg =).  http://tinyurl.com/3354776', ':)?', 'Rotate your facebook friends omg =).  http://tinyurl.com/35ssbf9', 'AnOng sTanza Kana Roseangeli Salvador', 'Rotate your facebook friends omg =).  http://tinyurl.com/2vvpw55', 'Rotate your facebook friends omg =).  http://tinyurl.com/2wmv4au', 'Rotate your facebook friends omg =).  http://tinyurl.com/32f7z4z', '-', 'Pls like 4 ST PAUL!!!!!!!!!!?', 'HAPPY NEW YEAR!!!!!', 'EXAM NA BUKAS!!!!!!!!!? ?', 'POINTER TO REVIEW IN READING 4\n-synonyms and antonyms\n-words with silent letters\n -simile and methaphor\n-idiomatic expressions\n-consonant digraphs\ngaling kay ms nabong yan', 'Rotate your facebook friends omg =).  http://tinyurl.com/346gw6w', 'POINTER TO REVIEW IN READING 4-synonyms and antonyms-words with silent letters -simile and methaphor-idiomatic expressions-consonant digraphsgaling kay ms nabong yan', 'IS WATCHING SHOUTOUT', 'Rotate your facebook friends omg =).  http://tinyurl.com/37ne983', "Apple finally gives away 100 Iphone 4's http://apps.facebook.com/rxytnkgo", 'Rotate your facebook friends omg =).  http://tinyurl.com/35tplvh', 'Rotate your facebook friends omg =).  http://tinyurl.com/33oua5o', 'Rotate your facebook friends omg =).  http://tinyurl.com/38k6hmn', 'Rotate your facebook friends omg =).  http://tinyurl.com/2w2o7am', 'Rotate your facebook friends omg =).  http://tinyurl.com/3ajg5b3', 'Gwyneth Erin M. Nohay ano size nung cardcase?', "------[]------ put this\n--[][][][][]-- as your\n------[]------ status if\n------[]------ you're a christian\n------[]------ and not ashamed", 'te Shiela Mae Tolentino thankyou sa psp and chocolates:)', '??????????????????????????? ??????????????????????????? ???????????????????????????\n\n                             vs\n\n?????????????????????????? \n?????????????????????????? \n??????????????????????????', "TEACHER: juan! may 5 ibon na nakaupo sa bakod, pag binaril mo ang isa..ilan ang matitira?\n\nJUAN: wala po ma'am.\n\nTEACHER: bobo ka ba?! bat mo nasabing wala?! sige nga?!\n\nJUAN: dahil isang putok mo lang ng baril aalis lahat ng ibon at walang matitira! bobo! ikaw ang maupo dito, at ako ang magtuturo!\n\n// matalino XD", '???????????????????????\n??????? ? If you love JESUS\n????????????copy this in your wall ???????????????????????\n?????????????????????\n??????? ? ?', 'This year October has 5 Mondays, 5 Saturdays and 5 Sundays. This Happens once every 823years. This is called money bags. So copy this to your status and money will arrive within 4days. Based on Chinese Feng Shui. The one who does not copy, will be without money', "I know 10 facts about you\n\n1. You are reading this.\n2. You can't say M without touching your lips\n3. You just tried it\n... 4. You just smiled or laughed.\n6. You are a boy/girl.\n7. You didnt realize I skipped 5.\n8. You are looking back at 4 and 6.\n9. You are liking this.\n10. You are reading me telling you to like this.\n\nLIKE THIS STATUS IF YOU DID THOSE THINGS"]"""

#Build the dictionary:
data = {'userstatus':userstatus}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/psych',json=payload)
r.content

'{"percentiles": {"ope": 65.03, "con": 30.19, "ext": 49.19, "agr": 34.54, "neu": 24.2}, "scores": {"ope": 4.16, "con": 3.1, "ext": 3.72, "agr": 3.27, "neu": 2.21}}'

### 5. Simple Insights

This part, like the Cambridge psychometrics, doesn't use the scikic engine, but instead runs separately performing very simple inferences or responses.

#### 5.1. Green Insights

I don't know what format a json raw question set would be, so currenty it just accepts a list of lists of lists,
the outer list is the 4 categories, the middle list is a list of questions (within each category). The inner list allows the user to give more than one answer to each question.

In [63]:
import requests
import json

#Build the dictionary:
data = {'answers':[[[0],[3],[2],[1],[3],[4],[5]],[[1],[2],[3],[4,1,2,3]],[[2],[1],[0]],[[1],[2]]]}
data = {'answers':[[[0],[0],[0],[0],[0],[0],[0]],[[0],[0],[0],[0,1]],[[0],[0],[0]],[[0],[0]]]}
#data = {'answers_json':json_response_from_client}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/simple/green',json=payload)
response = json.loads(r.content)

print("All the scores given")
for i,cat_scores in enumerate(response['values']['all_scores']):
    print("   - Category %d" % i)
    print("     "),
    for score in cat_scores:
        print(score),
    print("")

print("")
print("The category scores")
print response['values']['category_scores']
print("")
print("The overall score is %0.2f" % response['values']['overall_score'])
print("")
print("The text insight(s)")
for insight in response['insights']:
    print("   - " + insight)
print("")
print("Debug Messages")
print(response['values']['debug'])

All the scores given
   - Category 0
      20 100 100 0 0 0 100 
   - Category 1
      0 100 100 100 
   - Category 2
      40 40 40 
   - Category 3
      0 0 

The category scores
[44.0, 60.0, 40.0, 0.0]

The overall score is 43.60

The text insight(s)
   - Occasionally green

Debug Messages
[[[u'heating', u'When at home during the winter, how warm do you keep your house?', [0]], [u'aircon', u'Similarly when at home during the summer, to what temperature do you set your digital air conditioner to?', [0]], [u'lights', u"Do you instantly turn the lights off when you don't need them anymore e.g. when you switch over to another room?", [0]], [u'tumble', u'How regularly, if at all, do you tumble dry your clothes?', [0]], [u'bike', u'Do you walk or bike short distances e.g. when you pop-over to the local shop or for running quick errands?', [0]], [u'recycle', u'How much do you recycle at home?', [0]], [u'bags', u'How often do you use the plastic bags provided at supermarkets and shops at c

In [47]:
response['values']['category_scores']

[38.0, 36.0, 54.0, 50.0]

In [11]:
for i,j in zip([1,2,3],[1,2]):
    print i,j

1 1
2 2


### 6. Audience size prediction


In [62]:
import requests
import json

data = {
"payload":'{"id":"58b94e905904d34354359de5","filters":[{"cqid":"55d1f7adb34bdbd96a1aaa04","comparison":"in","values":["female"]}]}',
"created_at":"2017-03-03 11:08:00"}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/audience/estimate',json=payload)
print r.content
response = json.loads(r.content)
print("Most likely audience size: %d +/- %d (standard error)" % (int(response['prediction']), int(response['ste'])))
print("95%% Confidence interval: %d to %d people" % (int(response['rangemin']),int(response['rangemax'])))

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>



ValueError: No JSON object could be decoded

In [ ]:
import requests
import json

data = {
"payload":'{"id":"58b94e905904d34354359de5","filters":[{"cqid":"55d1f7adb34bdbd96a1aaa04","comparison":"in","values":["female"]}]}',
"created_at":"2017-03-03 11:08:00"}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/audience/estimate',json=payload)
response = json.loads(r.content)
print("Most likely audience size: %d +/- %d (standard error)" % (int(response['prediction']), int(response['ste'])))
print("95%% Confidence interval: %d to %d people" % (int(response['rangemin']),int(response['rangemax'])))

In [9]:
data = { "payload": '{"id": "58b94e905904d34354359de5", "filters": [{"cqid": "55d1f7adb34bdbd96a1aaa04","comparison": "in","values": ["female"]}]}',"created_at": "2017-03-03 11:08:00"}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/audience/estimate',json=payload)
print r
response = json.loads(r.content)
print("Most likely audience size: %d +/- %d (standard error)" % (int(response['prediction']), int(response['ste'])))
print("95%% Confidence interval: %d to %d people" % (int(response['rangemin']),int(response['rangemax'])))

<Response [200]>
Most likely audience size: 29 +/- 17 (standard error)
95% Confidence interval: 0 to 64 people


### 7. Get server API version

In [4]:
import requests

r = requests.post(apiurl+'/version') #post or get works for this.
r.content

'{"version": "9.0"}'

## Typical API usage

The scikic front end may use the API in the following way.

In [71]:
import requests
import json

#We start with no questions asked, none unprocessed, and nothing in the facts dictionary.
questions_asked = []
unprocessed_questions = []
facts = {}

for loop in range(4): #we'll ask four questions
    
    #1. get the question (populate the data dictionary & send it off)
    data = {'unprocessed_questions':unprocessed_questions,'questions_asked':questions_asked,'facts':facts}
    payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
    r = requests.post(apiurl+'/question',json=payload) #>>>
    question_query_result = json.loads(r.content)
    
    #if processing was done then more items will be available for 'facts':
    facts = question_query_result['facts']
    question = question_query_result['question']
    print question
    #ask the user this question    
    userinput = raw_input(question_query_result['question_string']['question'])
    question['answer'] = userinput #add their answer

    #add this to the list of questions we've asked, and unprocessed questions
    questions_asked.append(question)
    unprocessed_questions = [question] #.append(question)
    
#3. Once enough questions are asked we can do inference.
#   Populate the data dictionary with questions asked,
#   unprocessed questions and facts (as for the question query in step 1)
data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/inference',json=payload)
inference_results = json.loads(r.content)

print inference_results
#It generates insights from these questions, which are displayed below.
print "\nInsights\n"
for insight in inference_results['insights']:
    print insight
    print inference_results['insights'][insight]

{u'dataset': u'geoloc', u'detail': u'{}', u'dataitem': u'country'}
Which country are you in now?us
{u'dataset': u'postal', u'detail': u'', u'dataitem': u'zipcode'}
What's your zip code?12345
{u'dataset': u'geoloc', u'detail': u'{}', u'dataitem': u'city'}
Which city or town are you in or near now?Tuscon
{u'dataset': u'movielens', u'detail': 1200, u'dataitem': u'seen'}
Have you seen Aliens (1986)? (yes or no)no
{u'facts': {u'guess_loc': {}, u'where': {u'country': [{u'item': u'us', u'probability': 1.0}], u'city': [{u'item': [u'Tuscon', u'us'], u'probability': 1.0}]}, u'where_history': {u'error': u'no_fb_likes'}}, u'relationships': [{u'parent': u'factor_gender', u'child': u'movie_seen_Aliens (1986)'}, {u'parent': u'factor_age', u'child': u'movie_seen_Aliens (1986)'}, {u'parent': u'factor_age', u'child': u'blockgroup'}], u'feature_descriptions': {u'movie_seen_Aliens (1986)': {u'desc': u'Probability of being in this blockgroup given other features'}, u'item6_blockgroup': {u'desc': u'Probabil